In [83]:
# !pip install -q pybotters numpy pandas
# !pip install -q python-dotenv


## Idea
* バブル相場ではどうせ価格は上がるんだから、一時的に下げた後のお得なところで買えば勝てるよね！

## 設計構想

* 直近N分の価格取得
  * 最低価格を計算


* 

***
## TODO
### 最優先
* volumeの自動取得
* order sizeの算出
* priceをformatする機能
* closing orderのsizeをpositionから算出

### 次点
* 複数銘柄対応

***

In [1]:
import pybotters
import pandas as pd
from pandas import DataFrame

In [2]:
import math

def adjust_price(price, price_tick):
    round_price = max(0, int(-math.log10(price_tick)))
    adjusted_price = round((price // price_tick) * price_tick, round_price)
    return int(adjusted_price) if adjusted_price.is_integer() else adjusted_price

def adjust_qty(qty, qty_step, min_qty):
    round_qty = max(0, int(-math.log10(qty_step)))
    adjusted_qty = max(round((qty // qty_step) * qty_step, round_qty), min_qty)
    return int(adjusted_qty) if adjusted_qty.is_integer() else adjusted_qty

In [6]:
class Exchange():
    """
    取引所と直接APIのやり取りを行う
    """
    def __init__(self, client):
        self.client = client
        pass
    
    async def get_klines(self, pair_symbol:str, interval:str='60', limit:str='5') -> pd.DataFrame:
        """
        /v5/market/kline
        ローソク足(kline)を取得
        """
        async with self.client.get(
            f"/v5/market/kline?category=linear&interval=60&symbol={pair_symbol}&interval={interval}&limit={limit}"
        ) as resp:
            content = await resp.json()

        klines = content['result']['list']
        klines = pd.DataFrame(klines)
        klines.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'turnover']

        return klines

    async def get_latest_price(self, pair_symbol:str):
        """"""""
        klines = await self.get_klines(pair_symbol, interval='1', limit='1')
        latest_price = float(klines['close'].values[-1])
        return latest_price

    async def get_order(self, pair_symbol:str):
        async with self.client.get(
            f"/v5/order/realtime?category=linear&symbol={pair_symbol}&limit=50"  # max limit == 50
        ) as resp:
            content = await resp.json()
        orders = content['result']['list']
        # print(f'exchange.get_order: {orders}')
        return orders


    async def get_symbol_info(self, pair_symbol):
        """銘柄の情報を取得"""
        async with self.client.get(f"/v5/market/instruments-info?category=linear&symbol={pair_symbol}") as resp:
            content = await resp.json()
            symbol_info = content['result']
            price_tick = float(symbol_info['list'][0]['priceFilter']['tickSize'])
            qty_step = float(symbol_info['list'][0]['lotSizeFilter']['qtyStep'])
            return (price_tick, qty_step)

    async def get_balance_info(self, coin: str='USDT') -> float:
        """残高を取得"""
        async with self.client.get(f"/v5/account/wallet-balance?accountType=UNIFIED&coin={coin}") as resp:
            content = await resp.json()
            # print('[exchange]-get_balance:', content)
            return content['result']['list']

    async def get_position(self):
        async with self.client.get("/v5/position/list?category=linear&settleCoin=USDT") as resp:
            content = await resp.json()
            symbol_list = [{'symbol': json['symbol'], 'side': json['side'], 'size': json['size']} for json in content['result']['list']]
            return symbol_list

    async def set_leverage(self, pair_symbol:str, leverage:int):
        """
        https://bybit-exchange.github.io/docs/v5/position/leverage
        """
        data = {
            'category': "linear", 
            'symbol': pair_symbol,
            'buyLeverage': leverage,
            'sellLeverage': leverage,
        }
        async with self.client.post("/v5/position/set-leverage", data=data) as resp:
            content = await resp.json()
            # print('[exchange]-set_leverage:', content)
            return content['retMsg']

    async def create_order(self, pair_symbol, qty, side='Buy', price=None, reduce_only=False):
        data = data={
                'category': "linear",
                'symbol': pair_symbol,
                'side': side,
                'orderType': 'Limit',
                'price': price,
                'qty': qty,
                'reduceOnly': reduce_only,
        }
        async with self.client.post("/v5/order/create", data=data) as resp:
            content = await resp.json()
            if content['retCode'] == 0:
                print('[exchange]-create_order:', content['result'])
            return content['retMsg']

    async def cancel_all_orders(self, pair_symbol):
        data = data={'category': "linear", 'symbol': pair_symbol}
        async with self.client.post("/v5/order/cancel-all", data=data) as resp:
            content = await resp.json()
            if content['retCode'] == 0:
                print('[exchange]-cancel_all_orders:', content['result'])
            return content['retMsg']

class TradingStrategy():
    """
    ロジックをもとに売買に関わるシグナルを作成する
    """
    def __init__(self, exchange: Exchange, trading_volume: float=10, rate_of_drop:float=0.2, rate_of_pump:float=0.2, leverage: int=5):
        self.exchange = exchange
        self.trading_volume = trading_volume
        self.rate_of_drop = rate_of_drop
        self.rate_of_pump = rate_of_pump
        self.leverage = leverage
        self.symbol_info = {}
    
    async def initalize_setting(self, pair_symbol: str):
        # cancel all existing orders
        await self.exchange.cancel_all_orders(pair_symbol)

        # set leverage
        ret_msg = await self.exchange.set_leverage(pair_symbol, self.leverage)
        if ret_msg == 'OK':
            print(f'[TradingStrategy]-initalize_setting: leverage is set to {self.leverage}x')
        elif ret_msg == 'leverage not modified':
            print(f'[TradingStrategy]-initalize_setting: leverage is already {self.leverage}x')

        balance_info = await self.exchange.get_balance_info('USDT')
        usdt_balance = float(balance_info[0]['coin'][0]['availableToWithdraw'])
        print('[TradingStrategy]-initalize_setting:', usdt_balance)

        self.trading_volume = min(self.trading_volume, usdt_balance * int(self.leverage)) * 0.95
        print(f'[TradingStrategy]-initalize_setting: trading_volume is set to {self.trading_volume}')

        price_tick, qty_step = await self.exchange.get_symbol_info(pair_symbol)
        self.symbol_info = {'price_tick': price_tick, 'qty_step': qty_step}

    def calc_position_size(self):
        pass

    async def update_order(self):
        """
        update existing order
        """
        pass

    async def get_open_order(self, pair_symbol:str):
        """
        get existing order
        """
        orders = await self.exchange.get_order(pair_symbol)
        return orders

    async def calc_buy_price(self, pair_symbol:str, interval:str=60) -> float:
        klines:DataFrame = await self.exchange.get_klines(pair_symbol=pair_symbol, interval=interval, limit='5')
        high:float = klines['high'].astype(float).values
        highest_price:float = high.max()
        buy_price:float = highest_price * (1 - self.rate_of_drop)
        # レバレッジ部分は追加実装する必要あり
        return buy_price

    async def calc_sell_price(self, pair_symbol:str, interval:str=60) -> float:
        klines:DataFrame = await self.exchange.get_klines(pair_symbol=pair_symbol, interval=interval, limit='5')
        low:float = klines['low'].astype(float).values
        lowest_price:float = low.min()
        sell_price:float = lowest_price * (1 + self.rate_of_pump)
        return sell_price

    async def create_opening_order(self, pair_symbol) -> None:
        buy_price = await self.calc_buy_price(pair_symbol)
        adjusted_price = adjust_price(buy_price, self.symbol_info['price_tick'])
        adjusted_qty = adjust_qty(self.trading_volume / buy_price, self.symbol_info['qty_step'], 0.01)

        ret_msg = await self.exchange.create_order(pair_symbol=pair_symbol, qty=str(adjusted_qty), side='Buy', price=str(adjusted_price), reduce_only=False)
        if not ret_msg == 'OK':
            print(f'[TradingStrategy]-create_opening_order: {pair_symbol} {adjusted_qty} {adjusted_price} {ret_msg}')

    async def create_closing_order(self, pair_symbol) -> None:
        sell_price = await self.calc_sell_price(pair_symbol)
        adjusted_price = adjust_price(sell_price, self.symbol_info['price_tick'])
        adjusted_qty = adjust_qty(self.trading_volume / sell_price, self.symbol_info['qty_step'], 0.01)

        ret_msg = await self.exchange.create_order(pair_symbol=pair_symbol, qty=adjusted_qty, side='Sell', price=str(adjusted_price), reduce_only=True)
        if not ret_msg == 'OK':
            print(f'[TradingStrategy]-create_closing_order: {pair_symbol} {adjusted_qty} {adjusted_price} {ret_msg}')

class KaitenBot():
    """
    ロジックに沿ってトレードを行う最上位クラス
    """
    def __init__(self, client, strategy_params:dict):
        self.exchange = Exchange(client)
        self.trading_strategy = TradingStrategy(
            self.exchange,
            strategy_params['trading_volume'],
            strategy_params['rate_of_drop'],
            strategy_params['rate_of_pump'],
            strategy_params['leverage']
        )
    
    async def run(self, pair_symbol):

        # 0. cancel all orders
        await self.trading_strategy.initalize_setting(pair_symbol)

        # 1. check position
        positions = await self.exchange.get_position()
        current_position = None
        for position in positions:
            if position['symbol'] == pair_symbol:
                current_position = position
                print('current_position:', current_position)
                break

        # 2. create_order
        if current_position is None:
            # ポジションがない場合：新規ポジションを開く
            await self.trading_strategy.create_opening_order(pair_symbol)
        else:
            # ポジションがある場合：既存のポジションをクローズする
            await self.trading_strategy.create_closing_order(pair_symbol)

        order = await self.trading_strategy.get_open_order(pair_symbol)
        if order:
            print(f'There is {len(order)} orders in {pair_symbol}')
        else:
            print(f'There is no orders in {pair_symbol}')



In [7]:
import os
from dotenv import load_dotenv

load_dotenv('.env')

apis = {
    'bybit': [os.getenv('API_KEY'), os.getenv('API_SECRET')]
}

strategy_parmas = {
    'trading_volume': float(os.getenv('volume')), 
    'leverage': os.getenv('leverage'),
    'rate_of_drop': float(os.getenv('rate_of_drop')),
    'rate_of_pump': float(os.getenv('rate_of_pump'))
}

pair_symbol = os.getenv('pair_symbol')

print('strategy_parmas:', strategy_parmas, '\n')

async with pybotters.Client(apis=apis, base_url='https://api.bybit.com') as client:
    kaiten_bot = KaitenBot(client, strategy_parmas)
    await kaiten_bot.run(pair_symbol)


strategy_parmas: {'trading_volume': 10.0, 'leverage': '2', 'rate_of_drop': 0.25, 'rate_of_pump': 0.1} 

[exchange]-cancel_all_orders: {'list': [{'orderId': '9eb099d7-fb3f-41ae-b80e-11db5f433378', 'orderLinkId': ''}], 'success': '1'}
[TradingStrategy]-initalize_setting: leverage is already 2x
[TradingStrategy]-initalize_setting: 5.05451878
[TradingStrategy]-initalize_setting: trading_volume is set to 9.5
[exchange]-create_order: {'orderId': 'd99eae5d-a767-4096-9093-dfa34fa56ad1', 'orderLinkId': ''}
There is 1 orders in WIFUSDT


In [11]:
import os
from dotenv import load_dotenv

load_dotenv('.env')

apis = {
    'bybit': [os.getenv('API_KEY'), os.getenv('API_SECRET')]
}

pair_symbol = 'BNBUSDT'

async with pybotters.Client(apis=apis, base_url='https://api.bybit.com') as client:
    exchange = Exchange(client)
    symbol_info = await exchange.get_symbol_info(pair_symbol)
    print(symbol_info)


{'category': 'linear', 'list': [{'symbol': 'BNBUSDT', 'contractType': 'LinearPerpetual', 'status': 'Trading', 'baseCoin': 'BNB', 'quoteCoin': 'USDT', 'launchTime': '1624951080000', 'deliveryTime': '0', 'deliveryFeeRate': '', 'priceScale': '2', 'leverageFilter': {'minLeverage': '1', 'maxLeverage': '50.00', 'leverageStep': '0.01'}, 'priceFilter': {'minPrice': '0.05', 'maxPrice': '99999.90', 'tickSize': '0.05'}, 'lotSizeFilter': {'maxOrderQty': '3075.00', 'minOrderQty': '0.01', 'qtyStep': '0.01', 'postOnlyMaxOrderQty': '3075.00', 'maxMktOrderQty': '1300.00'}, 'unifiedMarginTrade': True, 'fundingInterval': 480, 'settleCoin': 'USDT', 'copyTrading': 'both', 'upperFundingRate': '0.0075', 'lowerFundingRate': '-0.0075'}], 'nextPageCursor': ''}


In [71]:
price_tick = float(symbol_info['list'][0]['priceFilter']['tickSize'])
qty_step = float(symbol_info['list'][0]['lotSizeFilter']['qtyStep'])
min_qty = float(symbol_info['list'][0]['lotSizeFilter']['minOrderQty'])

print(price_tick, qty_step, min_qty)

0.05 0.01 0.01


In [74]:
price = 334.45678901234
qty = 11111111111.11111111111
adjusted_price = adjust_price(price, price_tick)
adjusted_qty = adjust_qty(qty, qty_step, min_qty)
print(adjusted_price, adjusted_qty)

price_tick, qty_step, min_qty = (10, 0.0001, 0.001)
adjusted_price = adjust_price(price, price_tick)
adjusted_qty = adjust_qty(qty, qty_step, min_qty)
print(adjusted_price, adjusted_qty)

330 11111111111.1111
330 11111111111.1111


330 11111111111.1111


## 参考

[バブル相場用の回転ボットのコンセプトとQuantZoneロジック(追記あり)](https://note.com/hht/n/n63022edc4610)

* 「レバレッジL倍で、直近N分高値から、R %価格が下がった位置にエントリー指値」
* 「直近N分安値から、R %価格が上がったら位置に決済指値」